# Analizando archivos `.kml` de Google Maps

Este _Jupyter notebook_ se realizó con el fin de dar un tutorial al procesamiento de los datos recabados
en el sitio web https://infogram.com/verificado19s-1gxop4g94ejypwy, el cual contiene un mapa de la Ciudad
de México con datos **verificados** sobre edificios, centros de acopio y necesidades surgidas a partir
del terremoto del 19/09/2017.

Los datos pueden ser descargados en un archivo de extensión `.kml`, el cual no es más que un `.xml`.

Para analizar la sintaxis del documento, utilizo el paquete _BeautifulSoup_, que
a su vez hace uso de _lxml_. Las documentaciones de ambos están en:
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- http://lxml.de/parsing.html

In [188]:
import codecs
import re

from bs4 import BeautifulSoup as BS
from pprint import pprint
from utils.utils import parse_extendeddata

In [5]:
# Import data
examplefile = 'examples/edificios.kml'
data = None
with codecs.open(examplefile, 'r') as f:
    data = f.read().strip()
soup = BS(data, 'lxml')

Se identifican **4 categorías** de objetos.
Los datos están organizados en un árbol con la siguiente estructura (curada):

```
Daños y derrumbes
  +-- <tipo_daño_1>
  +-- <description> (str)
  +-- <ubicación>
  +----- <lat>
  +----- <long>
  +-- <data_frame> (dict-like)
  +----- <k1>
  +------- v1
  +----- <k2>
  +------- v2
        .
        .
        .
  +-- <tipo_daño_2>
        .
        .
        .
Centros de Acopio
  +-- <nombre_centro_1>
  +-- <description> (str)
  +-- <ubicación>
  +----- <lat>
  +----- <long>
  +-- <data_frame> (dict-like)
  +----- <k1>
  +------- v1
  +----- <k2>
  +------- v2
        .
        .
        .
  +-- <nombre_de_centro_2>
        .
        .
        .
Albergues Oficiales
  +-- <nombre_albergue_1>
  +-- <description> (str)
  +-- <ubicación>
  +----- <lat>
  +----- <long>
  +-- <data_frame> (dict-like)
  +----- <k1>
  +------- v1
  +----- <k2>
  +------- v2
        .
        .
        .
  +-- <nombre_albergue_2>
        .
        .
        .
datos.csv
  +-- <dato_1>
  +-- <description> (str)
  +-- <ubicación>
  +----- <lat>
  +----- <long>
  +-- <data_frame> (dict-like)
  +----- <k1>
  +------- v1
  +----- <k2>
  +------- v2
        .
        .
        .
  +-- <dato_2>
        .
        .
        .
```
Al parecer la última jerarquía `datos.csv` se refiere a sucesos que no pudieron ser acomodados en alguna de las tres jerarquías anteriores.

In [196]:
# Useful data are subtrees under the tag 'folder'
map_info = soup.find_all('folder')

# Keep track of each element in the list separately
danyos = None
acopios = None
albergues = None
dcsv = None
for info in map_info:
    info_name = info.find('name').text
    if info_name == 'Daños y derrumbes':
        danyos = info
    elif info_name == 'Centros de Acopio':
        acopios = info
    elif info_name == 'Albergues Oficiales':
        albergues = info
    elif info_name == 'datos.csv':
        dcsv = info

En cada jerarquía, hay una lista de objetos con una descripción asociada. Para ilustrar esto,
el siguiente _snippet_ imprimirá los _primeros_ detalles encontrados bajo la categoría
`'Daños y derrumbes'`. Generalmente, el nodo `description` contiene un compilado de `data_frame`
en forma de `str`.

In [207]:
# Sample exploration of the 'Daños y derrumbes' subtree structure
print('categoría:', danyos.find('name').text, end='\n--\n')
danyos_list = danyos.find_all('placemark')
print('contiene', len(danyos_list), 'objetos', end='\n\n')

fst_danyo = danyos_list[0]
print('Primer objeto:', end='\n\n  --\n  ')
print('tipo de daño:', fst_danyo.find('name').text, end='\n  --\n  ')
print('descripción:', fst_danyo.description.text, end='\n  --\n  ')
print('data_frame:', end='\n  ')
data_frame = parse_extendeddata(fst_danyo.extendeddata)
pprint(data_frame, indent=2)

categoría: Daños y derrumbes
--
contiene 76 objetos

Primer objeto:

  --
  tipo de daño: Daño Mayor
  --
  descripción: entrecalles: Sur 20 y sur 16calle: Oriente 229numero_exterior: 379atrapados: lesionados: desaparecidos: fallecidos: rescatados: link_google_maps: https://goo.gl/maps/r9Qh6fcV7yF2lon: -99.0845049000lat: 19.3943899000entidad: Integrara iztacalcotipo_reporte: ]]>
  --
  data_frame:
  { 'atrapados': '',
  'calle': 'Oriente 229',
  'colonia': 'Agrícola Oriental',
  'desaparecidos': '',
  'entidad': 'Integrara iztacalco',
  'entrecalles': 'Sur 20 y sur 16',
  'fallecidos': '',
  'lat': '19.3943899000',
  'lesionados': '',
  'link_google_maps': 'https://goo.gl/maps/r9Qh6fcV7yF2',
  'lon': '-99.0845049000',
  'lugar': 'residencial',
  'numero_exterior': '379',
  'rescatados': '',
  'tipo_reporte': ''}


La función `parse_extendeddata` se encuentra definida en [`utils.py`](utils/utils.py).

## En resumen,

Cabe destacar que
- para obtener una lista de objetos por cada categoría hay que hacer uso del método 
`.findall('placemark')`;
- una vez tenida dicha lista, para cada uno de sus elementos hay un diccionario que puede
  ser obtenido pasándole a la función `parse_extendeddata` un nodo `soup` delimitado por
  `extendeddata`.

Por último, cabe destacar que la ubicación exacta de un lugar (latitud y longitud) no siempre
se encuentra en un data frame. A continuación describo algunas maneras de encontrarla:

In [226]:
# Ways to obtain latitude and logitude
lat, long = data_frame['lat'], data_frame['lon']
print(lat, long)

long, lat = fst_danyo.point.coordinates.text.strip().split(',')[:-1]
print(lat, long)

datum = dcsv.find_all('placemark')[1]
lat, long = datum.address.text.strip().split()
print(lat, long)

19.3943899000 -99.0845049000
19.3943899 -99.0845049
19.4184045 -99.1610117219
